In [1]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

In [2]:
from Hybrid import *
from Recommenders.SLIM.SLIMElasticNetRecommender import *
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
## SLIM
slim = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train)
slim.fit(topK=8894, l1_ratio=0.05565733019999427, alpha=0.0012979360257937668, workers=7)
## PURESVD
pure_svd = PureSVDRecommender(URM_train)
pure_svd.fit(num_factors=70, random_seed=42)
## P3ALPHA
p3alpha = P3alphaRecommender(URM_train)
p3alpha.fit(topK=76, alpha=0.377201600381895, normalize_similarity=True)
## RP3BETA
rp3beta = RP3betaRecommender(URM_train)
rp3beta.fit(topK=101, alpha=0.3026342852596128, beta=0.058468783118329024)
## USER KNN
userknn = UserKNNCFRecommender(URM_train)
userknn.fit(topK=529, shrink=45, similarity='asymmetric', normalize=True,
                                       feature_weighting='TF-IDF')
## ITEM KNN
itemknn = ItemKNNCFRecommender(URM_train)
itemknn.fit(topK=31, shrink=435, similarity='tversky', normalize=True,
                                       feature_weighting='BM25', tversky_alpha=0.17113169506422393, tversky_beta=0.5684024974085575)
## EASE_R
#ease_r = EASE_R_Recommender(URM_train)
#ease_r.fit(topK=24, l2_norm=37.54323189430143)

model = LinearHybridRecommender

from Evaluation.Evaluator import EvaluatorHoldout

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

SLIMElasticNetRecommender: URM Detected 458 ( 3.6%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 330 ( 1.5%) items with no interactions.


100%|█████████▉| 22192/22222 [01:34<00:00, 81.94it/s] 

PureSVDRecommender: URM Detected 458 ( 3.6%) users with no interactions.
PureSVDRecommender: URM Detected 330 ( 1.5%) items with no interactions.
PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing SVD decomposition... done in 1.06 sec
P3alphaRecommender: URM Detected 458 ( 3.6%) users with no interactions.
P3alphaRecommender: URM Detected 330 ( 1.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 6014.77 column/sec. Elapsed time 3.69 sec
RP3betaRecommender: URM Detected 458 ( 3.6%) users with no interactions.
RP3betaRecommender: URM Detected 330 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 4705.11 column/sec. Elapsed time 4.72 sec
UserKNNCFRecommender: URM Detected 458 ( 3.6%) users with no interactions.
UserKNNCFRecommender: URM Detected 330 ( 1.5%) items with no interactions.
Similarity column 12638 (100.0%), 9752.10 column/sec. Elapsed time 1.30 sec
ItemKNNCFRecommender: URM

100%|█████████▉| 22216/22222 [01:49<00:00, 81.94it/s]

Similarity column 22222 (100.0%), 8815.30 column/sec. Elapsed time 2.52 sec
EvaluatorHoldout: Ignoring 2570 (20.3%) Users that have less than 1 test interactions


In [25]:
import itertools
import optuna as op

# Map the names to the recommender objects
# Map the names to the recommender objects
recommender_dict = {
    'slim': slim,
    'puresvd': pure_svd,
    'p3alpha': p3alpha,
    'rp3beta': rp3beta,
    'userknn': userknn,
    'itemknn': itemknn
}

# Generate all possible combinations of recommender names
all_combinations = []
for r in range(1, len(recommender_dict) + 1):
    all_combinations.extend(itertools.combinations(recommender_dict.keys(), r))

# Convert combinations to string format
all_combinations_str = ['_'.join(comb) for comb in all_combinations]

def objective(trial):
    # Select a combination from the pre-defined list
    selected_combination_str = trial.suggest_categorical('selected_combination', all_combinations_str)

    # Convert the string back to the original recommender objects
    selected_combination = [recommender_dict[name] for name in selected_combination_str.split('_')]

    # Dynamically generate weights for the selected recommenders
    weights = [trial.suggest_float(f'weight_{i}', 0, 1) for i in range(len(selected_combination))]
    
    # Create hybrid recommender with the suggested weights
    hybrid_recommender = model(URM_train, selected_combination)
    hybrid_recommender.fit(weights)
    
    # Evaluate the hybrid recommender
    result, _ = evaluator_validation.evaluateRecommender(hybrid_recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [29]:
best_params = {
    "alphas": 1,
}

study_name = "hybrid-allmodels-study"  # Unique identifier of the study.
storage_name = "sqlite:///db.db".format(study_name)
#op.delete_study(study_name, storage_name)
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize", load_if_exists=True)
#study.enqueue_trial(best_params)
study.optimize(objective, n_trials=50)

[I 2023-12-03 01:37:59,248] Using an existing study with name 'hybrid-allmodels-study' instead of creating a new one.


SLIMElasticNetP3alphaItemKNNCFHybridRecommender: URM Detected 458 ( 3.6%) users with no interactions.
SLIMElasticNetP3alphaItemKNNCFHybridRecommender: URM Detected 330 ( 1.5%) items with no interactions.
EvaluatorHoldout: Processed 10068 (100.0%) in 10.27 sec. Users per second: 981


[I 2023-12-03 01:38:09,884] Trial 300 finished with value: 0.0314276767189462 and parameters: {'selected_combination': 'slim_p3alpha_itemknn', 'weight_0': 0.9577175308652484, 'weight_1': 0.7567721297949973, 'weight_2': 0.6500067609306452}. Best is trial 243 with value: 0.03146035923970984.


SLIMElasticNetP3alphaItemKNNCFHybridRecommender: URM Detected 458 ( 3.6%) users with no interactions.
SLIMElasticNetP3alphaItemKNNCFHybridRecommender: URM Detected 330 ( 1.5%) items with no interactions.
EvaluatorHoldout: Processed 10068 (100.0%) in 9.92 sec. Users per second: 1015


[I 2023-12-03 01:38:19,926] Trial 301 finished with value: 0.03143414464183251 and parameters: {'selected_combination': 'slim_p3alpha_itemknn', 'weight_0': 0.9587099986473953, 'weight_1': 0.7966249994543064, 'weight_2': 0.6456520427511198}. Best is trial 243 with value: 0.03146035923970984.


...

[I 2023-12-03 01:45:45,009] Trial 347 finished with value: 0.031456756752495915 and parameters: {'selected_combination': 'slim_p3alpha_itemknn', 'weight_0': 0.9383739710960782, 'weight_1': 0.8325740404685683, 'weight_2': 0.7083829457658045}. Best is trial 243 with value: 0.03146035923970984.


RP3betaUserKNNCFItemKNNCFHybridRecommender: URM Detected 458 ( 3.6%) users with no interactions.
RP3betaUserKNNCFItemKNNCFHybridRecommender: URM Detected 330 ( 1.5%) items with no interactions.
EvaluatorHoldout: Processed 10068 (100.0%) in 10.87 sec. Users per second: 926


[I 2023-12-03 01:45:56,026] Trial 348 finished with value: 0.023093030882065484 and parameters: {'selected_combination': 'rp3beta_userknn_itemknn', 'weight_0': 0.4981301961246196, 'weight_1': 0.8366596398356081, 'weight_2': 0.7263036795703376}. Best is trial 243 with value: 0.03146035923970984.


PureSVDRP3betaUserKNNCFItemKNNCFHybridRecommender: URM Detected 458 ( 3.6%) users with no interactions.
PureSVDRP3betaUserKNNCFItemKNNCFHybridRecommender: URM Detected 330 ( 1.5%) items with no interactions.
EvaluatorHoldout: Processed 10068 (100.0%) in 11.52 sec. Users per second: 874


[I 2023-12-03 01:46:07,706] Trial 349 finished with value: 0.023210328496383388 and parameters: {'selected_combination': 'puresvd_rp3beta_userknn_itemknn', 'weight_0': 0.9071822450261121, 'weight_1': 0.8133892246543677, 'weight_2': 0.7080451008992801, 'weight_3': 0.2795835322741268}. Best is trial 243 with value: 0.03146035923970984.


In [30]:
study.best_params

{'selected_combination': 'slim_p3alpha_itemknn',
 'weight_0': 0.9156068194320912,
 'weight_1': 0.817561194412642,
 'weight_2': 0.6931684679976798}

In [31]:
final = model(URM_train_validation, [slim, p3alpha, rp3beta, itemknn])
final.fit(alphas=[study.best_params["weight_0"],study.best_params["weight_1"], study.best_params["weight_2"]])

SLIMElasticNetP3alphaRP3betaItemKNNCFHybridRecommender: URM Detected 214 ( 1.7%) users with no interactions.
SLIMElasticNetP3alphaRP3betaItemKNNCFHybridRecommender: URM Detected 134 ( 0.6%) items with no interactions.


In [32]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users=[])
evaluator_test.evaluateRecommender(final)

EvaluatorHoldout: Ignoring 2181 (17.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Processed 10457 (100.0%) in 10.28 sec. Users per second: 1018


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10      0.087406                 0.146044  0.121361  0.044922    0.074536   
 
              MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                        ...   
 10      0.255372  0.136474  0.101622  0.486851       0.33478  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.827425          0.402833    0.827425       0.059011   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10           10.275576                   0.997986             0.169418   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                   

In [20]:
from Recommenders.NonPersonalizedRecommender import TopPop

top_recommender = TopPop(URM_all)
top_recommender.fit()
generateSubmission(final, top_recommender)

/Users/alessandro/RecSys-2023-polimi/Data_manager/UserUtils.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['item_list'] = submission['item_list'].apply(lambda x: ' '.join(map(str, x)))  # remove brakets
/Users/alessandro/RecSys-2023-polimi/Data_manager/UserUtils.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['item_list'] = submission['item_list'].astype(str)
